<a href="https://colab.research.google.com/github/Angelawork/AI4Good_LiteraLingo_M3Project/blob/main/FineTuning/T5_fine_tuning_Angela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm #gives a completion bar for your loop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [17]:
import transformers

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#There are 2 model:figurative-nlp/t5-figurative-paraphrase and figurative-nlp/sarcasm-hyperbole-humor-paraphrase
model_name1 = "figurative-nlp/t5-figurative-paraphrase"
model_name2 = "figurative-nlp/sarcasm-hyperbole-humor-paraphrase"

from transformers import AdamW

tokenizer = AutoTokenizer.from_pretrained(model_name1)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name1)
# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

In [ ]:
#model.eval()
#idioms_df = pd.read_csv('/content/Untitled Folder/Proverbs.csv',encoding = "ISO-8859-1")
#fig = list(idioms_df["Figurative sentence"][:100])
#lit = list(idioms_df["Literal meaning"][:100])
#input_ids = tokenizer("paraphrase the sentence : "+fig[0], padding=True, truncation=False,return_tensors="pt").input_ids
#outputs = model.generate(input_ids,num_beams = 5)
#result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#print("IN: ", fig[0])
#print("OUT: ", result)
#print("reference: ", lit[0])

# split dataset

In [18]:
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, AdamW
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd

df=pd.read_csv('/content/Simile.csv',encoding = "ISO-8859-1").sample(frac=1).reset_index(drop=True).drop_duplicates()[:1000]

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the test set further into validation and test sets
val_df, test_df = train_test_split(test_df, test_size=0.8, random_state=42)

# Print the sizes of the resulting datasets
print("total set size:", len(df))
print("Train set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

total set size: 298
Train set size: 238
Validation set size: 12
Test set size: 48


In [ ]:
# Set up device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
# Define a custom dataset for CSV data
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row['Figurative sentence']
        target_text = row['Literal meaning']

        # Tokenize the input and target text
        input_tokens = self.tokenizer.encode_plus(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_tokens = self.tokenizer.encode_plus(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_tokens['input_ids'].squeeze(),
            'attention_mask': input_tokens['attention_mask'].squeeze(),
            'target_ids': target_tokens['input_ids'].squeeze(),
            'target_attention_mask': target_tokens['attention_mask'].squeeze()
        }

In [19]:
# Define the dataset and data loader for training set
train_dataset = CustomDataset(train_df, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
!pip install bert-score
import bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git
import bleurt
from bleurt import score
bleurt_scorer = score.BleurtScorer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-n5jn44yw
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-n5jn44yw
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=2ea10088d43f9fe85e533e992eaa830c634c9b86d60069a7fae456d8865bee14
  Stored in directory: /tmp/pip-ephem-wheel-cache-e61sq00v/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())
    # Remove stop words and punctuation
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    # Return the preprocessed text as a string
    return ' '.join(tokens)

# Calculate cosine similarity for list of text objects
def calculate_cosine_similarity(text1, text2):
    # Preprocess the texts
    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)
    
    # Vectorize the texts
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text1, preprocessed_text2])
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    similarity_score = similarity_matrix[0][0]
    
    return similarity_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Set training parameters
from torch.nn import L1Loss, MSELoss
l1_strength = 1e-5
l2_strength = 1e-5

num_epochs = 5
learning_rate = 1e-3
optimizer = AdamW(model.parameters(), lr=learning_rate)
bert_scores = []
bleurt_scores = []
similarity_scores = []

# Fine-tuning loop for training set
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        # Calculate L1 and L2 regularization
        l1_loss = 0
        l2_loss = 0
        for param in model.parameters():
            l1_loss += torch.sum(torch.abs(param))
            l2_loss += torch.sum(torch.square(param))
        regularization_loss = l1_strength * l1_loss + l2_strength * l2_loss
        loss += regularization_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)
        
        # Convert the predicted and target ids to text
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(target_ids, skip_special_tokens=True)
        
        # Calculate BERT score
        _, _, bert_score_f1 = bert_score.score(predicted_texts, target_texts, lang="en", verbose=False)
        bert_scores.append(round(bert_score_f1[0].tolist(),4))
        bleurt_scores.append(bleurt_scorer.score(references=target_texts, candidates=predicted_texts))
        for sentence1, sentence2 in zip(target_texts, predicted_texts):
          similarity_scores.append(calculate_cosine_similarity(sentence1, sentence2))

    accuracy = np.mean(bert_scores)
    accuracy2 = np.mean(bleurt_scores)
    accuracy3=np.mean(similarity_scores)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss}, Accuracy (BERT score): {accuracy}, Accuracy (Bleurt score): {np.mean(accuracy2)},cosine-similarity:{accuracy3}')


    

In [ ]:
# Define the dataset and data loader for validation set
val_dataset = CustomDataset(val_df, tokenizer, max_length=512)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Evaluation loop for validation set
model.eval()
total_val_loss = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f'Validation Loss: {avg_val_loss}')

Validation Loss: 0.3205801049868266


# test

In [ ]:
# Define the dataset and data loader for test set
test_dataset = CustomDataset(test_df, tokenizer, max_length=512)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Evaluation loop for test set
model.eval()
total_test_loss = 0
bert_scores = []
bleurt_scores = []
similarity_scores=[]

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids, decoder_attention_mask=target_attention_mask)
        loss = outputs.loss

        total_test_loss += loss.item()

        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)
        
        # Convert the predicted and target ids to text
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(target_ids, skip_special_tokens=True)
        
        # Calculate BERT score
        _, _, bert_score_f1 = bert_score.score(predicted_texts, target_texts, lang="en", verbose=False)
        bert_scores.append(round(bert_score_f1[0].tolist(),4))
        bleurt_scores.append(bleurt_scorer.score(references=target_texts, candidates=predicted_texts))
        for sentence1, sentence2 in zip(target_texts, predicted_texts):
          similarity_scores.append(calculate_cosine_similarity(sentence1, sentence2))
        print(f"input_texts: {input_texts} predicted_texts: {predicted_texts}, target_texts: {target_texts}")

    accuracy = np.mean(bert_scores)
    accuracy2 = np.mean(bleurt_scores)
    accuracy3=np.mean(similarity_scores)
    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f'Test Loss: {avg_test_loss}, Accuracy (BERT score): {accuracy}, Accuracy (Bleurt score): {np.mean(accuracy2)}, cosine-similarity:{accuracy3}')

# Simile

input_texts: ['It was a moonless nights, the air was still and the crickets were as if on vacation.', 'The library grew back like a persistent garden.', 'They put up a brave face as governments become more like a dystopia.', 'After a few weeks he starting to feel like an outcast.'] 
## predicted_texts: ['It was a moonless nights, the air was still still the crickets were still', 'The library grew back prettily', 'They put up a brave face as governments become more authoritarian', 'After a few weeks he starting to feel strangea little lonely'], 

## target_texts: ['It was a moonless nights, the air was still and the crickets were silent', 'The library grew back prettily', 'They put up a brave face as governments become increasingly authoritarian', 'After a few weeks he starting to feel a little lonely']

Test Loss: 0.014054735058120318, Accuracy (BERT score): 0.975542857142857, Accuracy (Bleurt score): 0.07601729811972412, cosine-similarity:0.7844800646923608



# Sarcasm
input_texts: ['You love to waste my time?', 'The Romans tell fabulous tales of my bath...... and handmaidens...... and my morals.', "I've got it, tony!", "That's great, a sex offender at a library."] 

predicted_texts: ['You you like towasting my time?', 'The Romans of telling mea tale... my tales and talemais... and morals...', 'I have it, to!', 'That are  lotsexex at at  library.'], 

target_texts: ['Do you like wasting my time?', 'The Romans are telling unreal stories about my spa, my maids, my morality.', 'I got her, Tony.', 'We gota sex offender in the library.']
Test Loss: 0.06965618977944056, Accuracy (BERT score): 0.8886833333333333, Accuracy (Bleurt score): -0.3538652488961816, cosine-similarity:0.3221476036837769

# Idioms

##input_texts: ['She is a very nice person and is always willing to go the extra mile to help others.', 'Early to bed will give you a head start in the morning.', "I've heard that they change the crown jewels by their dead ringers so that they are protected from thieves and is that true?", 'My daughter was anything but a sound sleeper, she kept me awake all night most of the first year.'] 
##predicted_texts: ['She is a very nice person and is always willing to go to nice person go others.', 'Early to bed will give you   to in the morning.', "I've heard that they change that crownerss by their  by that they are protected from thieves and is that they?", 'My daughter was anything  a sound sleeper, she kept me awake all night most of the first year.'], 
##target_texts: ['She is a very nice person and is always willing to try very hard to help others.', 'Early to bed will give you an early start in the morning.', "I've heard that they change the crown jewels with fakes so that they are protected from thieves and is that true?", 'My daughter was certainly not a sound sleeper, she kept me awake all night most of the first year.']
Test Loss: 0.07475833209852377, Accuracy (BERT score): 0.9343766666666666, Accuracy (Bleurt score): 0.07886044414093098, cosine-similarity:0.6029421899681924

#Metonymies
input_texts: ['the guest tasted the bottle', 'media figures roundly condemned the magazine', 'the whole table laughed at his joke', 'new age synths surged in my ears'] 
predicted_texts: ['the guest tasted the water', 'media figures round condemned condemned the article', 'the whole table laughed at his joke', 'new age synths surged in my music']

target_texts: ['the guest tasted the wine', 'media figures roundly condemned the publisher', 'the whole table laughed at his people', 'new age synths surged in my beat']


Test Loss: 0.013414590735919774, Accuracy (BERT score): 0.966025, Accuracy (Bleurt score): 0.018013737299629757, cosine-similarity:0.7749545318832657

# metaphor
nput_texts: ["The girl had the laughter of a loon's warble", "She captured all the men's hearts", "The girl's friends were as flaky as a bar of iron", 'The speaker was as quiet as a newborn baby.'] 

predicted_texts: ['The girl was verying', "She captured all the men's hearts", "The girl's friends were fla fla", 'The speaker was very quiet.'], 

target_texts: ['The laughter is haunting.', "She had all the men's hearts", "The girl's friends were totally reliable.", 'The speaker was very quiet.']

Test Loss: 0.022819667764835887, Accuracy (BERT score): 0.9333488888888886, Accuracy (Bleurt score): -0.06580077600147989

# shuffled: 
input_texts: ['Daryl just takes off with Merle?', 'Who signs off on early release?', 'I looked at the remains that had been scattered over the table Like a bumpy red collage.', 'Of course, in those days, I was only a tea boy...'] 

predicted_texts: ['Daryl justs leaving leaving away Merle?', 'Who signss off off early release?', 'I looked at the remains that s red  on. over the table.', 'Of course, in was only a tea boy in in...'], 

target_texts: ["Daryl's just gone with Merl?", "Who's signing for early release?", 'I looked at the remains like lumpy red paint tossed across the table.', 'Of course, I was just a little boy back then...']

Test Loss: 0.07153916855653127, Accuracy (BERT score): 0.806888888888889, Accuracy (Bleurt score): 0.16305972784757614

##Hyperbole 

input_texts: ["Actually, it wasn't even English.", "Actually, I didn't even know him.", 'Safer than the palace, where I lived with a killer!', "We've got two people running."] 

## predicted_texts: ['Actually, it was not English. all.', 'I didn not know know him.', "Safe's safer than than in the palace, I lived with  killer!", 'We have two people running the ground.'], 

##target_texts: ['Indeed, it was not English at all.', 'I did not really know him.', "It's safer here than in the Palace where I lived with the killer!", 'We have two people on the run.']

Test Loss: 0.03903847262263298, Accuracy (BERT score): 0.9183133333333333, Accuracy (Bleurt score): 0.1165491214229001

# Save the fine-tuned model

In [ ]:
# Save the fine-tuned model
model.save_pretrained('T5model_Simile')
tokenizer.save_pretrained('T5tokenizer_Simile')

('T5tokenizer_Simile/tokenizer_config.json',
 'T5tokenizer_Simile/special_tokens_map.json',
 'T5tokenizer_Simile/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

source_path = "/content/T5model_Simile/pytorch_model.bin"  # Replace with the actual path to your file
target_path = "/content/drive/MyDrive/pytorch_model.bin"  # Replace with the desired path in your Google Drive

shutil.copyfile(source_path, target_path)

'/content/drive/MyDrive/pytorch_model1.bin'

# try fine-tuned model:

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#model_path = '/content/drive/MyDrive/T5model_Hyperbole_Angela'
#fine_tuned_model = torch.load(model_path)